In [1]:
#!unzip Text-Detoxification.zip

In [2]:
!pip install transformers
!pip install evaluate
!pip install accelerate -U
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.22.0
    Uninstalling accelerate-0.22.0:
      Successfully uninstalled accelerate-0.22.0


In [19]:
import torch
import sys
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig
import nltk
nltk.download('punkt')

sys.path.append('../')
kaggle = False
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    if os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive':
        kaggle = True
        
if kaggle:
    sys.path.append('/kaggle/input/text-detoxification/Text-Detoxification')



[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from src.data.make_dataset import TextDetoxificationDataset
from src.models.train_model import TrainTextDetoxificationModel

In [ ]:
file_path = 'data/raw/filtered.tsv'
if kaggle:
    file_path = '/kaggle/input/text-detoxification/Text-Detoxification/' + file_path
    
model_checkpoint = "eugenesiow/bart-paraphrase"
df = pd.read_csv(file_path, sep='\t')

config = AutoConfig.from_pretrained(model_checkpoint)
config.max_length = 64

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, config = config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
from sklearn.model_selection import train_test_split


# Split the dataset into train, test, and validation sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Create datasets using the split data
train_dataset = TextDetoxificationDataset(train_df.reset_index(drop=True), tokenizer, 60)
val_dataset = TextDetoxificationDataset(val_df.reset_index(drop=True), tokenizer, 60)
test_dataset = TextDetoxificationDataset(test_df.reset_index(drop=True), tokenizer, 60)

dataset = {"train": train_dataset, "validation":val_dataset, "test":test_dataset}

In [ ]:
logs = TrainTextDetoxificationModel(model, dataset, tokenizer, 1, 1e-4, 0.01, batch_size = 128)

In [21]:
torch.save(model.state_dict(), 'bart-paraphrase-finetuned.pth')
